In [ ]:
# Install Kaggle API
!pip install kaggle

# Import the necessary library and upload kaggle.json
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json

# Create the Kaggle directory and move the kaggle.json file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset (this downloads a ZIP file)
!kaggle datasets download -d humansintheloop/recycling-dataset

# Unzip the downloaded file
!unzip recycling-dataset.zip -d /content/


In [ ]:
import os
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical

# Paths to folders (adjust as needed)
json_folder = '/content/ds0/ann'
image_folder = '/content/ds0/img'

# Initialize lists to store data
image_data = []
labels = []

# Helper function to get labels from JSON files
def get_label_from_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
        for obj in data['objects']:
            for tag in obj['tags']:
                if tag['name'] == 'Material':
                    return tag['value']
    return None

# Process images and their corresponding labels
def process_images_and_labels(json_folder, image_folder):
    for json_filename in os.listdir(json_folder):
        if json_filename.endswith('.json'):
            json_path = os.path.join(json_folder, json_filename)

            # Get the label (Material)
            label = get_label_from_json(json_path)
            if label is None:
                continue  # Skip if no label is found

            # Get the corresponding image filename (remove ".json" to get the image file name)
            image_filename = json_filename[:-5]  # Remove the last 5 characters ('.json')
            image_path = os.path.join(image_folder, image_filename)

            # Check if the image file exists
            if os.path.exists(image_path):
                print(f"Processing image: {image_path}")

                # Load and process the image
                image = load_img(image_path, target_size=(224, 224))
                image = img_to_array(image) / 255.0  # Normalize image

                # Append image and label
                image_data.append(image)
                labels.append(label)
            else:
                print(f"Image not found for {json_filename}: {image_path}")

# Run the processing
process_images_and_labels(json_folder, image_folder)

# Convert lists to numpy arrays
X = np.array(image_data)
y = np.array(labels)




In [ ]:

#Check Lengths
len(X)
len(y)

#Find the Unique Labels
unique_labels = np.unique(y)
print("Unique labels:", unique_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the training labels
y_train_encoded = label_encoder.fit_transform(y_train)

# Transform the test labels
y_test_encoded = label_encoder.transform(y_test)

# Display the mappings
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

In [ ]:
import numpy as np
'aluminum', 'celluloseacetate', 'glass', 'metal', 'paper', 'plastic', 'polystyrene', 'rubber', 'tetrapak', 'textile'
# Count of images per class before removal
current_counts = {
    0: 322,
    1: 16,
    2: 186,
    3: 18,
    4: 596,
    5: 1083,
    6: 32,
    7: 8,
    8: 144,
    9: 51
}

# Specify how many samples to remove from class 5
samples_to_remove = 600

# Find the indices of class 5
class_5_indices = np.where(y_train_encoded == 5)[0]

# Check if we have enough samples to remove
if len(class_5_indices) < samples_to_remove:
    print(f"Not enough samples to remove from class 5. Available: {len(class_5_indices)}, Requested: {samples_to_remove}")
else:
    # Randomly select indices to remove
    np.random.seed(42)  # For reproducibility
    indices_to_remove = np.random.choice(class_5_indices, size=samples_to_remove, replace=False)

    # Create masks to keep only the indices not selected for removal
    mask = np.isin(np.arange(len(y_train_encoded)), indices_to_remove, invert=True)

    # Filter out the removed samples
    X_train_filtered = X_train[mask]
    y_train_filtered = y_train_encoded[mask]

# Check the new counts after removal
new_counts = np.unique(y_train_filtered, return_counts=True)
print("New counts after removing samples from class 5:")
for label, count in zip(new_counts[0], new_counts[1]):
    print(f"Label {label}: {count}")




In [ ]:
!pip install tensorflow
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0

# Load EfficientNetB0 model with pre-trained weights
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Create the new model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(len(unique_labels), activation='softmax')  # Assuming you have 10 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Now you can fit your model using X_augmented and y_augmented
history = model.fit(X_train_filtered, y_train_filtered, validation_data=(X_test, y_test_encoded), epochs=5, batch_size=32)

#Evaluate Accuracy
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {accuracy:.2f}')

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze the first 100 layers
    layer.trainable = False

# Recompile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Continue training
history = model.fit(X_train_filtered, y_train_filtered, validation_data=(X_test, y_test_encoded), epochs=200, batch_size=32)



In [ ]:
import matplotlib.pyplot as plt

# Function to visualize predictions
def visualize_predictions(X_test, y_train_encoded, model, label_encoder, num_images=10):
    # Make predictions on the test set
    predictions = model.predict(X_train)

    # Get the predicted classes by taking the argmax
    predicted_classes = np.argmax(predictions, axis=1)

    # Select a random sample of images to display
    random_indices = np.random.choice(20, size=num_images, replace=False)

    # Create a plot
    plt.figure(figsize=(15, 5))
    for i, index in enumerate(random_indices):
        plt.subplot(2, num_images // 2, i + 1)
        plt.imshow(X_train[index])
        plt.axis('off')

        # Get the actual and predicted labels
        actual_label = y_train_encoded[index]
        predicted_label = predicted_classes[index]

        # Decode the labels back to original names
        actual_label_name = label_encoder.inverse_transform([actual_label])[0]
        predicted_label_name = label_encoder.inverse_transform([predicted_label])[0]

        # Display the actual and predicted labels
        plt.title(f"Actual: {actual_label_name}\nPredicted: {predicted_label_name}")

    plt.tight_layout()
    plt.show()

# Call the function to visualize predictions
visualize_predictions(X_train, y_train_encoded, model, label_encoder, num_images=10)
